# Web Scraping with Python
아래의 내용은 책 **"파이썬으로 웹 크롤러 만들기(Web Scraping with Python)"**을 참고하여 작성했습니다.

## Chapter 1. 첫 번째 웹 스크래퍼
- `BeautifulSoup`은 잘못된 HTML을 수정하여 쉽게 탐색할 수 있는 XML 형식의 파이썬 객체로 변환한다. 

In [101]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

- 아래와 같은 예외문은 만들 땐 귀찮지만, 한번 만들어두면 빠르고 믿을 수 있는 크롤러를 만들 수 있다.

In [105]:
def getTitle(url):
  try:
    html = urlopen(url)
  except HTTPError as e: # 404, 500같은 에러가 발생하는 경우
    return None # None값을 반환
  try:
    bsObj = BeautifulSoup(html.read(), 'html.parser')
    title = bsObj.body.h1
  except AttributeError as e: 
    return None 
  return title

In [106]:
title = getTitle('http://www.pythonscraping.com/pages/page1.html')

In [107]:
if title == None:
  print('Title could not be found')
else:
  print(title)

<h1>An Interesting Title</h1>


## Chapter 2. 고급 HTML 분석
Q. 크롤링하려는 웹 페이지의 HTMl 구조가 너무 복잡하거나 지저분할 땐, 어떻게 해야 할까?
1. 해당 웹 페이지의 모바일 버전 사이트를 찾아본다.
2. 자바스크립트 파일에 숨겨진 정보를 찾아본다.
3. 중요한 정보는 페이지 타이틀에 있을 때가 대부분이지만, 종종 원하는 정보가 페이지 URL에 들어 있을 때도 있다.

In [130]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [147]:
html = urlopen('http://www.pythonscraping.com/pages/warandpeace.html')

In [148]:
bsObj = BeautifulSoup(html.read(), 'html.parser')

- 이 `BeautifulSoup` 객체에 `findAll` 함수를 쓰면 특정 태그에 들어 있는 텍스트만 **리스트 형태**로 추출할 수 있다.
- 또한 `get_text()` 함수를 사용하여 태그를 제외한 텍스트만 추출할 수 있다.
    - `get_text()` 함수는 텍스트 블록보다는 BeautifulSoup 객체에 사용하는 게 원하는 결과를 얻기가 훨씬 쉽다.
    - `get_text()` 함수는 항상 마지막, 즉 최종 데이터를 출력하거나 저장, 조작하기 직전에만 써야 한다.

In [153]:
nameList = bsObj.findAll('span', {'class' : 'green'})
for name in nameList:
  print(name.get_text())

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


### find()와 findAll()
- 이 두 함수를 사용하여 태그를 다양한 속성에 따라 쉽게 필터링 할 수 있음.
- 리스트 형태로 반환

In [157]:
# 하나의 태그만 추출
bsObj.findAll('h1')

[<h1>War and Peace</h1>]

In [158]:
# 여러 태그 추출
bsObj.findAll({'h1', 'h2', 'h3'})

[<h1>War and Peace</h1>, <h2>Chapter 1</h2>]

In [160]:
# 하나의 속성을 기준으로 추출
bsObj.findAll('span', {'class' : {'green', 'red'}})[:5]

[<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
 Buonapartes. But I warn you, if you don't tell me that this means war,
 if you still try to defend the infamies and horrors perpetrated by
 that Antichrist- I really believe he is Antichrist- I will have
 nothing more to do with you and you are no longer my friend, no longer
 my 'faithful slave,' as you call yourself! But how do you do? I see
 I have frightened you- sit down and tell me all the news.</span>,
 <span class="green">Anna
 Pavlovna Scherer</span>,
 <span class="green">Empress Marya
 Fedorovna</span>,
 <span class="green">Prince Vasili Kuragin</span>,
 <span class="green">Anna Pavlovna</span>]

### next_siblings()
- `next_siblings()` 함수는 테이블에서 데이터를 쉽게 수집할 수 있으며, 특히 테이블에 타이틀 행이 있을 때 유용하다.
- 출력결과는 첫 번째 행을 제외한 모든 행이다.
- `next_siblings()` 함수를 보완하는 `previous_siblings()` 함수도 있다.
    - 이 함수는 그 목록의 마지막에 있는 태그를 쉽게 선택할 수 있을 때 사용한다.
    - 이 두 개의 함수는 리스트가 아니라 태그 하나만 반환한다는 점을 빼면 똑같이 동작한다.

In [161]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [162]:
html = urlopen('http://www.pythonscraping.com/pages/page3.html')

In [163]:
bsObj = BeautifulSoup(html.read(), 'html.parser')

In [164]:
for sibling in bsObj.find('table', {'id' : 'giftList'}).tr.next_siblings:
  print(sibling)



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr